# 공휴일 정보 가져오기 #

In [8]:
import requests
from configparser import RawConfigParser    # .ini 형식의 파일에서 정보를 파싱해서 가져오기 위한 모듈
import pandas as pd
import xml.etree.ElementTree as et          # .xml 형식의 파일에서 정보를 파싱하기 위한 모듈

# data.org 사이트에서 API를 사용하기 위한 key를 가져오는 함수 
def get_config(config, api_name):
    config.read('..\..\..\.ignore\API.ini')
    if 'DATA_ORG' in api_name.upper():
        config_data = {
            'encode':config.get('DATA_ORG', 'encode').strip(),
            'decode':config.get('DATA_ORG', 'decode').strip()
            }
        return config_data

config = RawConfigParser()
apikey=get_config(config, 'DATA_ORG')

solYear_list = ['2023','2024']
solMonth_list = ['01','02','03','04','05','06','07','08','09','10','11','12']

all_hollyday_list = []

for year in solYear_list:
    for month in solMonth_list:
        callback_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
        url_params = {'solYear':year,
            'solMonth':month,
            'serviceKey':apikey['decode']}
        
        req = requests.get(callback_url,params=url_params)  # requests로 api 데이터를 받아옴
        xml_content = req.content.decode('utf-8')           # req에 저장된 byte 데이터를 문자열 데이터로 변환
        xml_root = et.fromstring(xml_content)               # xml 형식을 갖춘 문자열을 xml 객체로 변환
        # 키값을 이용하여 데이터 파싱하는 코드
        body = xml_root.find('body')
        items_element = body.find('items')
        if items_element is not None:
            for item in items_element.findall('item'):
                item_data = {}
                for child in item:
                    item_data[child.tag] = child.text
                all_hollyday_list.append(item_data)
            

In [9]:
df = pd.DataFrame(all_hollyday_list, index=None)
df.drop(columns=['dateKind', 'isHoliday', 'seq'], inplace=True)
df.to_csv('hollyday_2023-2024.csv', index=False, encoding='utf-8-sig')